[출처] https://wikidocs.net/24603

Kaggle에서 사용되었던 영화 데이터셋을 가지고 영화 추천 시스템을 만들어보겠습니다. 

TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템을 만들 수 있습니다.

다운로드 링크 : https://www.kaggle.com/rounakbanik/the-movies-dataset

In [1]:
import pandas as pd
import numpy as np

# Data에 예외 제거 
data = pd.read_csv('movies_metadata.csv', error_bad_lines=False)

print(data.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# 데이터의 형태 확인
display(data.head(2))

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [0]:
# 훈련 데이터의 양을 줄이고 학습을 진행하고 싶은 경우, 변경하여 재저장
data = data[:3000]

In [0]:
# data의 overview 열에 Null 값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'].apply(lambda tfidf_matrix: np.str(tfidf_matrix)))

# overview에 대해서 tf-idf 수행
print(f"데이터 수, 단어 수: {tfidf_matrix.shape}")


데이터 수, 단어 수: (3000, 17186)


In [9]:
# 코사인 유사도를 사용한 문서의 유사도

from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(data.index, index=data['title']).drop_duplicates()
display(indices.head(50))
print()

idx = indices['Father of the Bride Part II']
print(idx)

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
Heat                               5
Sabrina                            6
Tom and Huck                       7
Sudden Death                       8
GoldenEye                          9
The American President            10
Dracula: Dead and Loving It       11
Balto                             12
Nixon                             13
Cutthroat Island                  14
Casino                            15
Sense and Sensibility             16
Four Rooms                        17
Ace Ventura: When Nature Calls    18
Money Train                       19
Get Shorty                        20
Copycat                           21
Assassins                         22
Powder                            23
Leaving Las Vegas                 24
Othello                           25
Now and Then                    


4


In [0]:
def get_recommendations(title, cosine_sim = cosine_sim):
  idx = indices[title]
  
  # 모든 영화에 대해서 해당 영화와의 유사도를 구한다. 
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 영화들을 정렬한다.
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 가장 유사한 10개의 영화를 받아온다.
  sim_scores = sim_scores[1:11]

  # 가장 유사한 10개의 영화의 인덱스를 받아온다.
  movie_indices = [i[0] for i in sim_scores]

  # 가장 유사한 10개의 영화의 제목을 리턴한다.
  return data['title'].iloc[movie_indices]

In [0]:
get_recommendations('Toy Story')

2997              Toy Story 2
1071    Rebel Without a Cause
3057          Man on the Moon
485                    Malice
1932                Condorman
448         For Love or Money
1032            The Sunchaser
2157        Indecent Proposal
3252          Bound for Glory
4078                Losin' It
Name: title, dtype: object